In [1]:
import torch
import os
import imageio

from model import *
from dataset import load_blender_data
from run import config_parser, create_nerf, render_path

import warnings
warnings.filterwarnings(action='ignore')

os.environ['CUDA_VISIBLE_DEVICES']='0'

Testing

In [2]:
parser = config_parser()
parser_str = '--config config_lego.txt --render_only'
args, _ = parser.parse_known_args(args=parser_str.split())
args.lindisp = False
render_kwargs_train, render_kwargs_test, start, grad_vars, optimizer = create_nerf(args)

Found ckpts ['./logs\\blender_paper_lego\\200000.tar']
Reloading from ./logs\blender_paper_lego\200000.tar
Not ndc!


In [3]:
render_kwargs_train['network_fn']

NeRF(
  (pts_linears): ModuleList(
    (0): Linear(in_features=63, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): Linear(in_features=319, out_features=256, bias=True)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): Linear(in_features=256, out_features=256, bias=True)
  )
  (views_linears): ModuleList(
    (0): Linear(in_features=283, out_features=128, bias=True)
  )
  (feature_linear): Linear(in_features=256, out_features=256, bias=True)
  (alpha_linear): Linear(in_features=256, out_features=1, bias=True)
  (rgb_linear): Linear(in_features=128, out_features=3, bias=True)
)

In [4]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
np.random.seed(0)

def render():

    parser = config_parser()
    parser_str = '--config config_lego.txt --render_only'
    args, _ = parser.parse_known_args(args=parser_str.split())
    args.lindisp = False
    
    print(args)

    # Load data
    K = None

    images, poses, render_poses, hwf, i_split = load_blender_data(args.datadir, args.half_res, args.testskip)
    print('Loaded blender', images.shape, render_poses.shape, hwf, args.datadir)
    i_train, i_val, i_test = i_split

    near = 2.
    far = 6.

    if args.white_bkgd:
        images = images[...,:3]*images[...,-1:] + (1.-images[...,-1:])
    else:
        images = images[...,:3]

    # Cast intrinsics to right types
    H, W, focal = hwf
    H, W = int(H), int(W)
    hwf = [H, W, focal]

    if K is None:
        K = np.array([
            [focal, 0, 0.5*W],
            [0, focal, 0.5*H],
            [0, 0, 1]
        ])

    # Create log dir and copy the config file
    basedir = args.basedir
    expname = args.expname
    os.makedirs(os.path.join(basedir, expname), exist_ok=True)
    f = os.path.join(basedir, expname, 'args.txt')
    with open(f, 'w') as file:
        for arg in sorted(vars(args)):
            attr = getattr(args, arg)
            file.write('{} = {}\n'.format(arg, attr))
    if args.config is not None:
        f = os.path.join(basedir, expname, 'config.txt')
        with open(f, 'w') as file:
            file.write(open(args.config, 'r').read())

    # Create nerf model
    render_kwargs_train, render_kwargs_test, start, grad_vars, optimizer = create_nerf(args)
    global_step = start

    bds_dict = {
        'near' : near,
        'far' : far,
    }
    render_kwargs_train.update(bds_dict)
    render_kwargs_test.update(bds_dict)

    # Move testing data to GPU
    render_poses = torch.Tensor(render_poses).to(device)

    # Short circuit if only rendering out from trained model
    print('RENDER ONLY')
    with torch.no_grad():
        # Default is smoother render_poses path
        images = None

        testsavedir = 'result'
        os.makedirs(testsavedir, exist_ok=True)
        print('test poses shape', render_poses.shape)
        
        # Rendering
        rgbs, _ = render_path(render_poses, hwf, K, args.chunk, render_kwargs_test, gt_imgs=images, savedir=testsavedir, render_factor=args.render_factor)
        print('Done rendering', testsavedir)
        imageio.mimwrite(os.path.join(testsavedir, 'video.mp4'), to8b(rgbs), fps=30, quality=8)

In [5]:
render()

Namespace(config='config_lego.txt', expname='blender_paper_lego', basedir='./logs', datadir='./data/nerf_synthetic/lego', netdepth=8, netwidth=256, netdepth_fine=8, netwidth_fine=256, N_rand=1024, lrate=0.0005, lrate_decay=500, chunk=32768, netchunk=65536, no_batching=True, no_reload=False, ft_path=None, N_samples=64, N_importance=128, perturb=1.0, use_viewdirs=True, i_embed=0, multires=10, multires_views=4, raw_noise_std=0.0, render_only=True, render_test=False, render_factor=0, precrop_iters=500, precrop_frac=0.5, dataset_type='blender', testskip=8, white_bkgd=True, half_res=True, lindisp=False, i_print=100, i_img=500, i_weights=10000, i_testset=50000, i_video=50000)
Loaded blender (138, 400, 400, 4) torch.Size([40, 4, 4]) [400, 400, 555.5555155968841] ./data/nerf_synthetic/lego
Found ckpts ['./logs\\blender_paper_lego\\200000.tar']
Reloading from ./logs\blender_paper_lego\200000.tar
Not ndc!
RENDER ONLY
test poses shape torch.Size([40, 4, 4])


  0%|          | 0/40 [00:00<?, ?it/s]

0 0.001993417739868164


  2%|▎         | 1/40 [00:09<05:53,  9.07s/it]

torch.Size([400, 400, 3]) torch.Size([400, 400])
1 9.0706946849823


...

38 9.216193199157715


 98%|█████████▊| 39/40 [05:50<00:09,  9.25s/it]

39 9.21917986869812


100%|██████████| 40/40 [06:00<00:00,  9.00s/it]


Done rendering result
